# Statistics in Python

Unlike R, there is no standard package for doing statistics in Python. For instance, NumPy already allows one to obtain summary statistics such as mean, standard, deviation and correlation coefficient. As of Aug 2018, [Statsmodels](http://www.statsmodels.org/stable/) seems to be quit popular. Perhaps its biggest advantage is that it has a set of API that supports R-style formulas via [patsy](https://patsy.readthedocs.io/en/latest/index.html).

In this notebook, we take a quick dive into the API that supports R-style formulas and perform some exploratory data analysis.  More examples can be found [here](http://www.statsmodels.org/stable/examples/index.html).

In [1]:
import statsmodels.formula.api as smf  # Import the API that supports R formulas
import pandas as pd
import numpy as np

We load the math scores data set in `mathscores.csv` and take a look at the first few records.

In [2]:
df = pd.read_csv('mathscores.csv')

df.head()

,ID,Section,Year,CGPA,Final.A,Final.B,Quizzes,Assignments
0,1,V,3,5.23,17,0.0,3.5,14.5
1,2,V,3,3.78,20,7.0,17.7,22.8
2,3,V,2,4.50,17,9.5,12.4,28.8
3,4,V,2,10.57,20,13.0,17.3,27.9
4,5,V,4,6.35,19,3.0,16.5,6.0


Each record consists of information for a student with a particular `ID`.  Each student's class section (either `'F'` or `'V'`), cumulative grade point average (CGPA), score for part A of the final exam, score for part B of the final exam, total over all term quizzes, and total over all term assignments are recorded.

We first create an additional column that contains the total scores of the final exam.

In [3]:
df['Final'] = df['Final.A'] + df['Final.B']
df.head()

,ID,Section,Year,CGPA,Final.A,Final.B,Quizzes,Assignments,Final
0,1,V,3,5.23,17,0.0,3.5,14.5,17.0
1,2,V,3,3.78,20,7.0,17.7,22.8,27.0
2,3,V,2,4.50,17,9.5,12.4,28.8,26.5
3,4,V,2,10.57,20,13.0,17.3,27.9,33.0
4,5,V,4,6.35,19,3.0,16.5,6.0,22.0


We can obtain the Pearson product-moment correlation coefficients on `Final` and `CGPA` as follows:

In [4]:
np.corrcoef(df['Final'], df['CGPA'])

array([[1.        , 0.74757844],
       [0.74757844, 1.        ]])

And the covariance matrix is obtained thus:

In [5]:
np.cov(df['Final'], df['CGPA'])

array([[101.5401404 ,  19.87115485],
       [ 19.87115485,   6.95816819]])

## Ordinary least squares

We now perform a linear regression with `Final` as the dependent variable and `CGPA` and `Quizzes` as independent variables.

In [6]:
fit = smf.ols(formula='Final ~ CGPA + Quizzes', data=df).fit()

As in R, we can obtain a summary.

In [7]:
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                  Final   R-squared:                       0.589
Model:                            OLS   Adj. R-squared:                  0.580
Method:                 Least Squares   F-statistic:                     69.38
Date:                Mon, 26 Nov 2018   Prob (F-statistic):           1.97e-19
Time:                        02:24:35   Log-Likelihood:                -328.01
No. Observations:                 100   AIC:                             662.0
Df Residuals:                      97   BIC:                             669.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.6760      2.426      2.752      0.0

To see just the fitted values, we do the following:

In [8]:
fit.fittedvalues[0:5]  # List the first five fitted values

0    20.876025
1    23.971430
2    23.246104
3    40.115204
4    29.595728
dtype: float64

To predict with the model, we need to set up a dictionary of NumPy one-dimensional arrays of the same length, one for each independent variable and then call `predict` from the fitted model.

In [9]:
# Specify two new observations for prediction.
# The first new observation has CGPA 3.4 and Quizzes score 4.5
# The second new observation has CGPA 9.0 and Quizzes score 12.3
newObs = {'CGPA': np.array([3.4,9.0]), 'Quizzes' : np.array([4.5,12.3])}

fit.predict(newObs) # Predict the final marks for the two observations specified in newObs

0    16.938634
1    34.021748
dtype: float64

We can see a list of attributes of the fitted model with the following:

In [10]:
dir(fit)

['HC0_se',
 'HC1_se',
 'HC2_se',
 'HC3_se',
 '_HCCM',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cache',
 '_data_attr',
 '_get_robustcov_results',
 '_is_nested',
 '_wexog_singular_values',
 'aic',
 'bic',
 'bse',
 'centered_tss',
 'compare_f_test',
 'compare_lm_test',
 'compare_lr_test',
 'condition_number',
 'conf_int',
 'conf_int_el',
 'cov_HC0',
 'cov_HC1',
 'cov_HC2',
 'cov_HC3',
 'cov_kwds',
 'cov_params',
 'cov_type',
 'df_model',
 'df_resid',
 'diagn',
 'eigenvals',
 'el_test',
 'ess',
 'f_pvalue',
 'f_test',
 'fittedvalues',
 'fvalue',
 'get_influence',
 'get_prediction',
 'get_robustcov_results',
 'initialize',
 'k_constant',
 'llf',
 'load',
 'model',


Most of the attributes that are functions have associated docstring.  For example, we can get info on `conf_int` by doing the following:

In [11]:
help(fit.conf_int)

Help on method conf_int in module statsmodels.regression.linear_model:

conf_int(alpha=0.05, cols=None) method of statsmodels.regression.linear_model.RegressionResultsWrapper instance
    conf_int(self, alpha=0.05, cols=None)
    
    Returns the confidence interval of the fitted parameters.
    
    Parameters
    ----------
    alpha : float, optional
        The `alpha` level for the confidence interval.
        ie., The default `alpha` = .05 returns a 95% confidence interval.
    cols : array-like, optional
        `cols` specifies which confidence intervals to return
    
    Notes
    -----
    The confidence interval is based on Student's t-distribution.



## Exercise

Obtain the adjusted R-squaredfor the model `fit.

In [12]:
# Your code here

Categorical variables must be within `C( )`.

In [13]:
f = 'Final ~ CGPA + C(Year)'

fit2 = smf.ols(formula=f, data=df).fit()
print(fit2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Final   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.544
Method:                 Least Squares   F-statistic:                     30.58
Date:                Mon, 26 Nov 2018   Prob (F-statistic):           2.37e-16
Time:                        02:24:35   Log-Likelihood:                -331.04
No. Observations:                 100   AIC:                             672.1
Df Residuals:                      95   BIC:                             685.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       10.4746      2.022      5.179   

One can see that `Year` is not statistically significant and one might want to drop it from the model.

## Exercise

Try to fit a variety of models with different choices of independent variables, keeping `Final` as the dependant variable.

In [14]:
# Your code here

## Logistic regression

We add a binary column `Pass` to our data frame so that 1 indicates a final exam score of 27 or above and 0 otherwise.

In [15]:
df['Pass'] = df['Final'].map(lambda x : 1 if x >= 27 else 0) # Here, we used a conditional expression
df.head()

,ID,Section,Year,CGPA,Final.A,Final.B,Quizzes,Assignments,Final,Pass
0,1,V,3,5.23,17,0.0,3.5,14.5,17.0,0
1,2,V,3,3.78,20,7.0,17.7,22.8,27.0,1
2,3,V,2,4.50,17,9.5,12.4,28.8,26.5,0
3,4,V,2,10.57,20,13.0,17.3,27.9,33.0,1
4,5,V,4,6.35,19,3.0,16.5,6.0,22.0,0


We now perform logistic regression with the formula `Pass ~ Quizzes + Assignments + C(Section)`.

In [16]:
f = 'Pass ~ Quizzes + Assignments + C(Section)'

logitfit = smf.logit(formula=f, data=df).fit()
print(logitfit.summary2())

Optimization terminated successfully.
         Current function value: 0.460543
         Iterations 6
                        Results: Logit
Model:              Logit            No. Iterations:   6.0000  
Dependent Variable: Pass             Pseudo R-squared: 0.301   
Date:               2018-11-26 02:24 AIC:              100.1085
No. Observations:   100              BIC:              110.5292
Df Model:           3                Log-Likelihood:   -46.054 
Df Residuals:       96               LL-Null:          -65.896 
Converged:          1.0000           Scale:            1.0000  
---------------------------------------------------------------
                 Coef.  Std.Err.    z    P>|z|   [0.025  0.975]
---------------------------------------------------------------
Intercept       -4.8343   1.2936 -3.7370 0.0002 -7.3698 -2.2988
C(Section)[T.V]  0.0297   0.5280  0.0562 0.9552 -1.0052  1.0645
Quizzes          0.2506   0.0830  3.0187 0.0025  0.0879  0.4132
Assignments      0.0761   0

Let us do some predictions with the model obtained.

In [17]:
ob1 = pd.DataFrame({'Quizzes': 17, 'Assignments':30, 'Section': 'V'}, index=[0])

logitfit.predict(ob1) # A pass should be predicted

0    0.850332
dtype: float64

In [18]:
ob2 = pd.DataFrame({'Quizzes': 7, 'Assignments':10, 'Section': 'F'}, index=[0])

logitfit.predict(ob2) # A fail should be predicted

0    0.089513
dtype: float64

## Exercise

Redo the logistic regression above replacing `Section` with `Year` and `Assignments` with `CGPA`.  Use the resulting model to obtain a predictions for the following:

| **Year** | **CGPA** | **Quizzes** |
|:--------:|:--------:|:-----------:|
|    1     |   8.9    |       13    |
|    2     |  10.3    |       20    |
|    3     |   5.8    |       9     |

The output of the predicted values should be:
```
0    0.838238
1    0.986977
2    0.325480
dtype: float64
```

In [19]:
# Your code here